# Test falsificación

La idea es asignar cada equipo a una categoría (fácil, regular, difícil) de forma aleatoria, y ver si existe el efecto asociado a la cantidad de partidos fáciles o difíciles

In [1]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
def CrearBaseEfectosFake(df_reg, df_liga, Torneos, dictequipos):
    dfs_efecto = []
    # Asignar la dificultad al azar
    for torneo in Torneos:
        dificiles = list(np.random.choice(dictequipos[torneo], 5, replace = False))
        remain = [e for e in dictequipos[torneo] if e not in dificiles]
        faciles = list(np.random.choice(remain, 5, replace = False))
        df_efecto = df_liga[df_liga['Torneo'] == torneo].reset_index(drop = True)
        for c in [3,4,5]:
            df_efecto['local_facil_fake_c%s' % c] = np.where(df_efecto['Local'].isin(faciles[:c]), 1, 0)
            df_efecto['local_dificil_fake_c%s' % c] = np.where(df_efecto['Local'].isin(dificiles[:c]), 1, 0)
            df_efecto['visita_facil_fake_c%s' % c] = np.where(df_efecto['Visita'].isin(faciles[:c]), 1, 0)
            df_efecto['visita_dificil_fake_c%s' % c] = np.where(df_efecto['Visita'].isin(dificiles[:c]), 1, 0)
        colsL = [k for k in df_efecto.columns if 'fake' in k and 'visita' in k]
        colsV = [k for k in df_efecto.columns if 'fake' in k and 'local' in k]
        colsOp = ['sum' for k in colsL]
        # Caso base r = 4
        groupL = df_efecto[df_efecto['Round'] <= 4].reset_index(drop = True).groupby('Local').agg(dict(zip(colsL,colsOp)))
        groupV = df_efecto[df_efecto['Round'] <= 4].reset_index(drop = True).groupby('Visita').agg(dict(zip(colsV,colsOp)))
        groupL.columns = [k.replace('visita','perc').replace('fake_', 'fake_p%s_' % 4) for k in colsL]
        groupV.columns = [k.replace('local','perc').replace('fake_', 'fake_p%s_' % 4) for k in colsV]
        group = (groupL + groupV)/4
        # Resto de casos
        for r in [5,6,7]:
            groupL_ = df_efecto[df_efecto['Round'] <= r].reset_index(drop = True).groupby('Local').agg(dict(zip(colsL,colsOp)))
            groupL_.columns = [k.replace('visita','perc').replace('fake_', 'fake_p%s_' % r) for k in colsL]

            groupV_ = df_efecto[df_efecto['Round'] <= r].reset_index(drop = True).groupby('Visita').agg(dict(zip(colsV,colsOp)))
            groupV_.columns = [k.replace('local','perc').replace('fake_', 'fake_p%s_' % r) for k in colsV]

            group_ = (groupL_ + groupV_)/r
            group = group.merge(group_, how = 'left', left_index = True, right_index = True)

        group = group.reset_index()
        group.columns = ['equipo' if x=='Local' else x for x in group.columns]
        group['Torneo'] = torneo
        dfs_efecto.append(group)
    df_efecto = df_reg.merge(pd.concat(dfs_efecto, ignore_index = True).reset_index(drop = True),
                             how = 'left',
                             left_on = ['equipo','Torneo'],
                             right_on = ['equipo', 'Torneo']).reset_index(drop = True)
    return df_efecto

In [3]:
def EstimacionEfectoFake(regdir, ligasdir, plotdir, liga, n_trials = 100):
    # Lee archivos
    file = '%s.xlsx' % liga
    df_reg = pd.read_excel(os.path.join(regdir,'Inglaterra.xlsx'))
    df_liga = pd.read_excel(os.path.join(ligasdir,'Inglaterra.xlsx'))
    
    # Crea lista con torneos de interés
    Torneos = df_reg['Torneo'].drop_duplicates().tolist()
    
    # Filtra la base de partidos según torneos de interés
    df_liga = df_liga[df_liga['Torneo'].isin(Torneos)].reset_index(drop = True)
    
    # Filtrar columnas de interés en ambos datasets
    dependants = [c for c in df_reg.columns if 'position' in c or 'points' in c]
    dependants_title = [c.replace('position','Posición').replace('points','Puntos').replace('_',' ') for c in dependants]
    dictependants = dict(zip(dependants, dependants_title))
    df_reg = df_reg[['equipo','Torneo'] + dependants]
    df_liga = df_liga[['Torneo','Round','Local','Visita']]
    
    # Crea diccionario con los equipos que participan en cada torneo
    dictequipos = {}
    for torneo in Torneos:
        dictequipos[torneo] = df_reg[df_reg['Torneo'] == torneo]['equipo'].tolist()
        
    # Filtrar base hasta el máximo de rondas consideradas (en este caso, 7)
    df_liga = df_liga[df_liga['Round'] <= 7].reset_index(drop = True)

    # Crear diccionario para almacenar p-valores
    dictpvalores = {}
    # Crear diccionario para almacenar nombres de graficos
    dictplotnames = {}
    # Crear diccionario para almacenar títulos de gráficos
    dictplottitles = {}
    for y in ['position_final','points_final']:
#     for y in dependants:
        for d in ['facil','dificil']:
            for l in [3,4,5]:
                for r in [4,5,6,7]:
                    # dificultad, r (partidos), l (corte)
                    x = 'perc_%s_fake_p%s_c%s' % (d, r, l)
                    X = 'Frac. %s primeros %s corte %s' % (d.replace('fa','fá').replace('fi','fí'),r,l)
                    Y = dictependants[y]
                    plotname = '%spartidos%sfaker%sl%s.png' %  (liga, d[0].lower(), r, l)
                    tupletitle = (X, Y, liga)
                    plottitle = 'Test de Falsificación\nVariable Independiente: %s\nVariable dependiente: %s\nLiga: %s' % tupletitle
                    dictpvalores[(y, x)] = []
                    dictplotnames[(y,x)] = plotname
                    dictplottitles[(y,x)] = plottitle
    for n in range(n_trials):
        print('Liga: %s, Iteración: %s     ' % (liga, n), end = '\t\r')
        # Genera base de datos de efecto fake
        df_efecto = CrearBaseEfectosFake(df_reg, df_liga, Torneos, dictequipos)
        # Ejecuta regresiones para una base de efectos fake y guarda los p-valores en el diccionario        
        for y in ['position_final','points_final']:
    #     for y in dependants:
            for d in ['facil','dificil']:
                for l in [3,4,5]:
                    for r in [4,5,6,7]:
                        x = 'perc_%s_fake_p%s_c%s' % (d, r, l)
                        formula = '%s ~ %s + equipo + Torneo' % (y,x)
                        mod = ols(formula = formula,
                                  data = df_efecto).fit(cov_type = 'cluster',
                                                        cov_kwds={'groups': df_efecto['equipo']})
                        dictpvalores[(y, x)].append(mod.pvalues[x])
    for key, value in dictpvalores.items():
        plt.hist(x=value, bins=20, alpha=0.7, rwidth=0.85)
        plt.grid(axis='y', alpha=0.75)
        plt.axvline(np.mean(value), color='r', linestyle='dashed', linewidth=1, label ='Media')
        plt.axvline(np.median(value), color='g', linestyle='dashed', linewidth=1, label = 'Mediana')
        plt.axvline(0.05, color='black', linestyle='dashed', linewidth=1, label = '0.05')
        plt.xlabel('Valores')
        plt.ylabel('Frecuencia')
        plt.legend()
        plt.title(dictplottitles[key])
        plt.savefig(os.path.join(plotdir,'%s' % dictplotnames[key]), bbox_inches = "tight", dpi = 250)
        plt.cla()
        plt.clf()
        plt.close('all')
    return dictpvalores

In [4]:
%%time
# Directorios
regdir = os.path.join(os.path.pardir, 'datos','regresiones')
ligasdir = os.path.join(os.path.pardir, 'datos','ligas')
plotdir = os.path.join(os.path.pardir, 'resultados','falsificacion-graficos')
# Lista de ligas:
# ligas = ['Alemania','Espana','Francia','Inglaterra','Italia']
liga = 'Inglaterra'
# EJECUCION
dictpvalores_ing = EstimacionEfectoFake(regdir, ligasdir, plotdir, liga, n_trials = 1000)

CPU times: user 1h 26min 43s, sys: 1min, total: 1h 27min 43s
Wall time: 42min 25s
